# Lecture 3: Spark Streaming

_Spark Streaming_ is an extension of the Spark API that enables scalabe stream processing.

The continous stream of input data can be ingested from many data sources such as **Kafka**, **Amazon s3** or **TCP sockets**. 

The Spark API allows to process data via high-level functions such as *map* and *reduce*. As we are going to see, it is also possible to use dataframe operations. 

Processed data can be exported to an external database and used to make live dashboards or offline analyses, or stored in files, or be used in a further stage of a Kafka pipeline. 

Overall, the practice of reading data from a set of sources, pre-process it, and then store it in a different format for later analysis is extremely common, and has its own name: **realtime ETL pipelines**.
- **E**xtract
- **T**transform
- **L**oad

Spark streaming works by dividing the input data into _micro-batches_ that can be treated as static datasets. In Spark this is referred to as a *discretized stream* (*DStream*). The DStream is represented using RDDs.

![DStream](imgs/lecture3/DStream.png)

Any transformation applied on the DStream, i.e. anything like a `Dstream.map()`, will act independently on each batch. For example, in the image below, we can filter the original RDD to remove some data and produce a new stream. 

![DStream_filter](imgs/lecture3/Dstream_filter.png)

In this lecture we will see how to setup a simple stream using a TCP socket as a data source.

## Create and Start a Spark Session

In [1]:
# import the python libraries to create/connect to a Spark Session
from pyspark.sql import SparkSession

# build a SparkSession 
#   connect to the master node on the port where the master node is listening (7077)
#   declare the app name 
#   configure the executor memory to 512 MB
#   either *connect* or *create* a new Spark Context
spark = SparkSession.builder \
    .master("spark://spark-master:7077")\
    .appName("My streaming spark application")\
    .config("spark.executor.memory", "512m")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
    .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/10 07:43:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
spark

In [2]:
# create a spark context
sc = spark.sparkContext

# print its status
sc

<SparkContext master=spark://spark-master:7077 appName=My streaming spark application>

## Spark _Streaming_ context

The first step of a Spark streaming application is the creation of a `StreamingContext`. 

The `StreamingContext` is a crucial component in Spark Streaming. It's responsible for initializing the Spark Streaming application and specifying how to handle micro-batches of data. 

The `StreamingContext` is a similar concept to the `sparkContext` but it requires to be initialized with some additional information to know how to handle the micro-batches.

To create a `StreamingContext`, you can use the `StreamingContext(SparkContext, batch_interval)` constructor. The `SparkContext` object provides the necessary environment for Spark Streaming, while the `batch_interval` parameter determines the (wall-time) duration of each batch in seconds.

It's important to note that you can only have at most **one** `StreamingContext` for each Spark application. Attempting to create multiple `StreamingContext` objects in a single application will result in errors.

Create a Spark `StreamingContext` with a batch interval of 2 seconds

In [3]:
from pyspark.streaming import StreamingContext

# create a streaming context with a batch interval of 2 seconds
ssc = StreamingContext(sc, 2) 

### Starting and Stopping Spark Streaming

To process data in real-time using Spark, we need to create a `StreamingContext`, define the operations to perform on the data, and specify the data source and sink to connect to.

Once the streaming operations are defined, we can start processing the stream by calling the `.start()` method of the `StreamingContext` object (`ssc` in our case). Similarly, we can stop the streaming processing by calling the `.stop()` method.

**NOTE:** It's important to note that when we stop the `StreamingContext`, the default behavior is to also stop the `SparkContext`. This means that the entire Spark application will be closed by default. To prevent this, we can pass the `stopSparkContext=False` option when stopping the `StreamingContext`.

### TCP Socket Source

For this example spark will read data from a TCP socket using Spark Streaming.

A TCP socket is a communication endpoint used to establish a connection between two devices over a network.
You can think of it as a telephone connection: two endpoints have to enstablish a connection; once the connection is enstablished, a communication can occur, with a data transfer; as soon as one of the two ends interrupts the connection the whole communcation is lost. 

We will generate a dummy data stream representing fake credit card transactions.

A simple python program will be used to create this data stream.
You will be able to find it in `utils/producer.py`. 
When executed, the producer will try to enstablish a TCP connection and send data on port `5555` of a given `host` (`spark-master` in our case). 

Before executing the producer program, take a moment to review the `producer.py` code to understand how it works. It's important to understand the logic of the program before using it to generate the streaming data.

In [4]:
! cat utils/producer.py

The producer will generate new records in the form of a random combination of:
- `name`
- `surname`
- `amount`: amount of the credit card transaction
- `delta_t`: time between transactions
- `flag`: random flag to indicate if potentially fraudolent or not

These information will be formatted into a `.json` data format

### Declaring the `StreamingContext` data source as a TCP socket

To inform Spark that the StreamingContext data source will be a TCP socket located at a specific `hostname` and `port`, we can use the `socketTextStream(hostname, port)` method.

Refer to the [StreamingContext documentation](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.streaming.StreamingContext.html) for additional available options.


In [5]:
# the hostname and port number
hostname = "spark-master"
portnumber = 5555

# declare the Spark Streaming source as TCP socket 
socket_stream = ssc.socketTextStream(hostname, portnumber)

### Start the python producer.py script

From a terminal/WSL, connect to the `spark-master` Docker container using the command
```bash
docker exec -it spark-master bash
``` 

From inside the docker container, move to the `/mapd-workspace` folder and execute the python script with the option `--hostname spark-master`:

```bash
python notebooks/utils/producer.py --hostname spark-master
```

## Exploring the data stream

The first thing we need to to is load the data describing each transaction, formatted as `json`.

In [9]:
import json

# use the map() transformation to apply the same function to all rdds
# the function we want to run is the json.load() of the messages
json_stream = socket_stream.map(lambda msg: json.loads(msg))

-------------------------------------------
Time: 2023-05-10 07:48:30
-------------------------------------------
{'name': 'John', 'surname': 'Johnson', 'amount': '836.46', 'delta_t': '4.13', 'flag': 0}
{'name': 'Jill', 'surname': 'Darby', 'amount': '664.54', 'delta_t': '7.28', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '554.90', 'delta_t': '5.87', 'flag': 1}
{'name': 'Alice', 'surname': 'Millers', 'amount': '121.85', 'delta_t': '4.26', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '976.95', 'delta_t': '9.79', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '505.05', 'delta_t': '8.97', 'flag': 0}
{'name': 'John', 'surname': 'Darby', 'amount': '845.74', 'delta_t': '0.22', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '4.21', 'delta_t': '9.83', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '807.85', 'delta_t': '8.93', 'flag': 1}
{'name': 'Joe', 'surname': 'Darby', 'amount': '347.08', 'delta_t': '0.43', 'flag': 0}
...



-------------------------------------------
Time: 2023-05-10 07:48:32
-------------------------------------------
{'name': 'Jill', 'surname': 'Millers', 'amount': '716.89', 'delta_t': '3.88', 'flag': 0}
{'name': 'John', 'surname': 'Darby', 'amount': '161.34', 'delta_t': '5.14', 'flag': 1}
{'name': 'Alice', 'surname': 'Jones', 'amount': '434.61', 'delta_t': '0.92', 'flag': 0}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '241.70', 'delta_t': '6.62', 'flag': 0}
{'name': 'Jill', 'surname': 'Jones', 'amount': '445.44', 'delta_t': '9.12', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '72.15', 'delta_t': '5.20', 'flag': 1}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '18.79', 'delta_t': '2.27', 'flag': 0}
{'name': 'Alice', 'surname': 'Smith', 'amount': '300.55', 'delta_t': '6.05', 'flag': 0}
{'name': 'Jill', 'surname': 'Smith', 'amount': '309.21', 'delta_t': '2.72', 'flag': 1}
{'name': 'Joe', 'surname': 'Smith', 'amount': '853.51', 'delta_t': '5.25', 'flag': 0}
...



-------------------------------------------
Time: 2023-05-10 07:48:34
-------------------------------------------
{'name': 'Andy', 'surname': 'Millers', 'amount': '681.72', 'delta_t': '0.74', 'flag': 1}
{'name': 'Andy', 'surname': 'Smith', 'amount': '191.08', 'delta_t': '5.22', 'flag': 0}
{'name': 'John', 'surname': 'Darby', 'amount': '294.89', 'delta_t': '2.51', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '377.84', 'delta_t': '8.36', 'flag': 0}
{'name': 'John', 'surname': 'Millers', 'amount': '574.99', 'delta_t': '3.51', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '204.37', 'delta_t': '1.54', 'flag': 1}
{'name': 'Joe', 'surname': 'Darby', 'amount': '916.36', 'delta_t': '0.12', 'flag': 1}
{'name': 'Joe', 'surname': 'Millers', 'amount': '331.47', 'delta_t': '3.94', 'flag': 0}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '171.59', 'delta_t': '6.58', 'flag': 0}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '149.55', 'delta_t': '7.59', 'flag': 0}
...



-------------------------------------------
Time: 2023-05-10 07:48:36
-------------------------------------------
{'name': 'Alice', 'surname': 'Johnson', 'amount': '871.71', 'delta_t': '1.95', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '941.54', 'delta_t': '0.16', 'flag': 0}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '266.47', 'delta_t': '7.20', 'flag': 1}
{'name': 'Joe', 'surname': 'Darby', 'amount': '615.34', 'delta_t': '6.58', 'flag': 0}
{'name': 'John', 'surname': 'Millers', 'amount': '573.97', 'delta_t': '4.13', 'flag': 0}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '483.65', 'delta_t': '0.31', 'flag': 1}
{'name': 'Jill', 'surname': 'Jones', 'amount': '95.92', 'delta_t': '7.41', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '997.60', 'delta_t': '9.52', 'flag': 0}
{'name': 'Andy', 'surname': 'Millers', 'amount': '483.50', 'delta_t': '2.87', 'flag': 1}
{'name': 'Joe', 'surname': 'Jones', 'amount': '66.43', 'delta_t': '0.73', 'flag': 0}
...



-------------------------------------------
Time: 2023-05-10 07:48:38
-------------------------------------------
{'name': 'Alice', 'surname': 'Johnson', 'amount': '356.42', 'delta_t': '8.28', 'flag': 1}
{'name': 'John', 'surname': 'Millers', 'amount': '61.48', 'delta_t': '9.53', 'flag': 0}
{'name': 'Jill', 'surname': 'Smith', 'amount': '846.88', 'delta_t': '2.86', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '519.23', 'delta_t': '4.08', 'flag': 0}
{'name': 'Alice', 'surname': 'Darby', 'amount': '235.67', 'delta_t': '3.09', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '290.30', 'delta_t': '4.85', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '245.44', 'delta_t': '0.41', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '444.40', 'delta_t': '0.68', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '696.49', 'delta_t': '3.99', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '670.86', 'delta_t': '6.12', 'flag': 

...

-------------------------------------------
Time: 2023-05-10 07:49:02
-------------------------------------------
{'name': 'Joe', 'surname': 'Johnson', 'amount': '100.59', 'delta_t': '0.61', 'flag': 0}
{'name': 'Andy', 'surname': 'Darby', 'amount': '233.41', 'delta_t': '8.57', 'flag': 1}
{'name': 'Jill', 'surname': 'Millers', 'amount': '238.39', 'delta_t': '8.10', 'flag': 1}
{'name': 'Andy', 'surname': 'Millers', 'amount': '389.13', 'delta_t': '1.00', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '640.34', 'delta_t': '6.64', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '155.82', 'delta_t': '1.64', 'flag': 1}
{'name': 'Andy', 'surname': 'Smith', 'amount': '135.82', 'delta_t': '6.12', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '868.67', 'delta_t': '9.64', 'flag': 0}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '129.02', 'delta_t': '3.75', 'flag': 0}
{'name': 'Jill', 'surname': 'Darby', 'amount': '936.92', 'delta_t': '9.53', 'flag': 0}
...



-------------------------------------------
Time: 2023-05-10 07:49:04
-------------------------------------------
{'name': 'John', 'surname': 'Millers', 'amount': '178.13', 'delta_t': '2.99', 'flag': 1}
{'name': 'Alice', 'surname': 'Jones', 'amount': '33.29', 'delta_t': '4.03', 'flag': 1}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '208.89', 'delta_t': '6.59', 'flag': 0}
{'name': 'Andy', 'surname': 'Millers', 'amount': '754.45', 'delta_t': '2.47', 'flag': 1}
{'name': 'John', 'surname': 'Darby', 'amount': '493.01', 'delta_t': '9.97', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '444.86', 'delta_t': '8.21', 'flag': 0}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '551.03', 'delta_t': '8.00', 'flag': 0}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '641.37', 'delta_t': '0.02', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '967.74', 'delta_t': '1.22', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '39.06', 'delta_t': '9.10', 'flag': 1}
...

-------------------------------------------
Time: 2023-05-10 07:49:06
-------------------------------------------
{'name': 'Joe', 'surname': 'Jones', 'amount': '171.59', 'delta_t': '0.89', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '354.59', 'delta_t': '7.32', 'flag': 0}
{'name': 'Joe', 'surname': 'Darby', 'amount': '200.98', 'delta_t': '1.94', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '643.57', 'delta_t': '3.58', 'flag': 1}
{'name': 'Jill', 'surname': 'Johnson', 'amount': '269.29', 'delta_t': '1.14', 'flag': 0}
{'name': 'John', 'surname': 'Darby', 'amount': '568.21', 'delta_t': '8.23', 'flag': 1}
{'name': 'John', 'surname': 'Darby', 'amount': '159.29', 'delta_t': '9.59', 'flag': 1}
{'name': 'Andy', 'surname': 'Jones', 'amount': '812.24', 'delta_t': '5.57', 'flag': 0}
{'name': 'Jill', 'surname': 'Darby', 'amount': '531.04', 'delta_t': '2.68', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '899.21', 'delta_t': '6.72', 'flag': 1}
...



-------------------------------------------
Time: 2023-05-10 07:49:08
-------------------------------------------
{'name': 'Jill', 'surname': 'Millers', 'amount': '920.67', 'delta_t': '2.69', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '218.56', 'delta_t': '6.77', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '969.02', 'delta_t': '0.83', 'flag': 0}
{'name': 'John', 'surname': 'Darby', 'amount': '733.68', 'delta_t': '7.29', 'flag': 0}
{'name': 'Joe', 'surname': 'Millers', 'amount': '766.26', 'delta_t': '1.54', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '270.60', 'delta_t': '1.26', 'flag': 1}
{'name': 'Jill', 'surname': 'Millers', 'amount': '453.65', 'delta_t': '2.40', 'flag': 1}
{'name': 'John', 'surname': 'Millers', 'amount': '871.99', 'delta_t': '2.52', 'flag': 1}
{'name': 'John', 'surname': 'Millers', 'amount': '64.15', 'delta_t': '0.66', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '420.19', 'delta_t': '0.91', 'flag': 0}
...



-------------------------------------------
Time: 2023-05-10 07:49:10
-------------------------------------------
{'name': 'Jill', 'surname': 'Millers', 'amount': '845.20', 'delta_t': '0.38', 'flag': 1}
{'name': 'Alice', 'surname': 'Darby', 'amount': '479.71', 'delta_t': '9.49', 'flag': 0}
{'name': 'Alice', 'surname': 'Darby', 'amount': '28.39', 'delta_t': '3.70', 'flag': 0}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '574.52', 'delta_t': '2.43', 'flag': 0}
{'name': 'Jill', 'surname': 'Johnson', 'amount': '928.02', 'delta_t': '9.03', 'flag': 0}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '898.52', 'delta_t': '0.09', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '736.36', 'delta_t': '6.00', 'flag': 0}
{'name': 'John', 'surname': 'Millers', 'amount': '948.51', 'delta_t': '5.26', 'flag': 0}
{'name': 'Andy', 'surname': 'Darby', 'amount': '423.25', 'delta_t': '7.33', 'flag': 0}
{'name': 'Jill', 'surname': 'Johnson', 'amount': '339.55', 'delta_t': '0.00', 'flag': 0}
..

It is possible to print some elements of each batch with `pprint()`. This can be used to explore the RDDs.

In [10]:
json_stream.pprint()

Py4JJavaError: An error occurred while calling None.org.apache.spark.streaming.api.python.PythonTransformedDStream.
: java.lang.IllegalStateException: Adding new inputs, transformations, and output operations after starting a context is not supported
	at org.apache.spark.streaming.dstream.DStream.validateAtInit(DStream.scala:226)
	at org.apache.spark.streaming.dstream.DStream.<init>(DStream.scala:67)
	at org.apache.spark.streaming.api.python.PythonDStream.<init>(PythonDStream.scala:224)
	at org.apache.spark.streaming.api.python.PythonTransformedDStream.<init>(PythonDStream.scala:241)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490

-------------------------------------------
Time: 2023-05-10 07:49:12
-------------------------------------------
{'name': 'Jill', 'surname': 'Millers', 'amount': '519.71', 'delta_t': '0.12', 'flag': 1}
{'name': 'Alice', 'surname': 'Smith', 'amount': '43.70', 'delta_t': '7.86', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '569.37', 'delta_t': '7.72', 'flag': 0}
{'name': 'Andy', 'surname': 'Darby', 'amount': '971.48', 'delta_t': '2.44', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '813.45', 'delta_t': '2.00', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '93.24', 'delta_t': '1.93', 'flag': 0}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '662.10', 'delta_t': '2.45', 'flag': 0}
{'name': 'Jill', 'surname': 'Millers', 'amount': '966.29', 'delta_t': '4.20', 'flag': 1}
{'name': 'Andy', 'surname': 'Smith', 'amount': '641.09', 'delta_t': '2.58', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '399.89', 'delta_t': '9.59', 'flag': 0}
...



-------------------------------------------
Time: 2023-05-10 07:49:14
-------------------------------------------
{'name': 'Alice', 'surname': 'Johnson', 'amount': '973.67', 'delta_t': '3.21', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '337.41', 'delta_t': '1.41', 'flag': 0}
{'name': 'Andy', 'surname': 'Darby', 'amount': '292.85', 'delta_t': '1.49', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '361.85', 'delta_t': '0.08', 'flag': 1}
{'name': 'Andy', 'surname': 'Millers', 'amount': '654.08', 'delta_t': '1.68', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '320.16', 'delta_t': '6.20', 'flag': 0}
{'name': 'John', 'surname': 'Darby', 'amount': '176.56', 'delta_t': '5.53', 'flag': 1}
{'name': 'Joe', 'surname': 'Smith', 'amount': '469.00', 'delta_t': '0.44', 'flag': 0}
{'name': 'John', 'surname': 'Millers', 'amount': '660.94', 'delta_t': '9.09', 'flag': 0}
{'name': 'Jill', 'surname': 'Darby', 'amount': '964.86', 'delta_t': '1.87', 'flag': 0}
...



-------------------------------------------
Time: 2023-05-10 07:49:16
-------------------------------------------
{'name': 'Alice', 'surname': 'Millers', 'amount': '553.25', 'delta_t': '1.42', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '576.58', 'delta_t': '9.93', 'flag': 1}
{'name': 'Jill', 'surname': 'Darby', 'amount': '399.44', 'delta_t': '6.91', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '576.85', 'delta_t': '0.79', 'flag': 0}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '72.04', 'delta_t': '1.00', 'flag': 0}
{'name': 'Joe', 'surname': 'Millers', 'amount': '988.11', 'delta_t': '0.46', 'flag': 0}
{'name': 'Joe', 'surname': 'Darby', 'amount': '404.63', 'delta_t': '4.81', 'flag': 1}
{'name': 'John', 'surname': 'Smith', 'amount': '800.73', 'delta_t': '7.49', 'flag': 0}
{'name': 'Jill', 'surname': 'Smith', 'amount': '565.91', 'delta_t': '7.49', 'flag': 0}
{'name': 'Joe', 'surname': 'Darby', 'amount': '952.86', 'delta_t': '8.21', 'flag': 1}
...



-------------------------------------------
Time: 2023-05-10 07:49:18
-------------------------------------------
{'name': 'John', 'surname': 'Jones', 'amount': '617.42', 'delta_t': '5.39', 'flag': 0}
{'name': 'Andy', 'surname': 'Darby', 'amount': '708.26', 'delta_t': '2.36', 'flag': 1}
{'name': 'Jill', 'surname': 'Johnson', 'amount': '754.04', 'delta_t': '0.63', 'flag': 0}
{'name': 'Jill', 'surname': 'Smith', 'amount': '765.12', 'delta_t': '6.58', 'flag': 0}
{'name': 'Jill', 'surname': 'Smith', 'amount': '570.52', 'delta_t': '7.41', 'flag': 0}
{'name': 'Jill', 'surname': 'Johnson', 'amount': '659.05', 'delta_t': '0.83', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '25.49', 'delta_t': '7.44', 'flag': 0}
{'name': 'Alice', 'surname': 'Darby', 'amount': '108.51', 'delta_t': '0.69', 'flag': 1}
{'name': 'John', 'surname': 'Johnson', 'amount': '536.09', 'delta_t': '6.33', 'flag': 0}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '760.87', 'delta_t': '6.63', 'flag': 0}
...



-------------------------------------------
Time: 2023-05-10 07:49:20
-------------------------------------------
{'name': 'Jill', 'surname': 'Smith', 'amount': '357.97', 'delta_t': '7.95', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '507.13', 'delta_t': '4.91', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '992.95', 'delta_t': '7.02', 'flag': 1}
{'name': 'Joe', 'surname': 'Millers', 'amount': '82.32', 'delta_t': '1.02', 'flag': 1}
{'name': 'Joe', 'surname': 'Darby', 'amount': '499.41', 'delta_t': '7.61', 'flag': 0}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '505.63', 'delta_t': '9.18', 'flag': 0}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '854.92', 'delta_t': '0.39', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '166.51', 'delta_t': '4.37', 'flag': 0}
{'name': 'Alice', 'surname': 'Darby', 'amount': '928.32', 'delta_t': '2.13', 'flag': 0}
{'name': 'Andy', 'surname': 'Darby', 'amount': '704.80', 'delta_t': '9.35', 'flag': 0}
...



-------------------------------------------
Time: 2023-05-10 07:49:22
-------------------------------------------
{'name': 'Andy', 'surname': 'Jones', 'amount': '771.56', 'delta_t': '0.04', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '249.58', 'delta_t': '1.06', 'flag': 0}
{'name': 'Joe', 'surname': 'Millers', 'amount': '581.94', 'delta_t': '1.53', 'flag': 0}
{'name': 'Andy', 'surname': 'Millers', 'amount': '781.04', 'delta_t': '9.31', 'flag': 0}
{'name': 'Joe', 'surname': 'Millers', 'amount': '490.52', 'delta_t': '3.59', 'flag': 0}
{'name': 'Alice', 'surname': 'Darby', 'amount': '226.98', 'delta_t': '1.76', 'flag': 0}
{'name': 'John', 'surname': 'Darby', 'amount': '296.06', 'delta_t': '9.52', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '711.45', 'delta_t': '4.89', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '41.76', 'delta_t': '7.27', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '170.22', 'delta_t': '8.37', 'flag': 1}
...



-------------------------------------------
Time: 2023-05-10 07:49:24
-------------------------------------------
{'name': 'Andy', 'surname': 'Johnson', 'amount': '937.38', 'delta_t': '2.07', 'flag': 0}
{'name': 'Joe', 'surname': 'Millers', 'amount': '330.11', 'delta_t': '0.31', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '399.69', 'delta_t': '6.09', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '213.45', 'delta_t': '3.36', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '748.11', 'delta_t': '7.11', 'flag': 0}
{'name': 'Andy', 'surname': 'Darby', 'amount': '679.11', 'delta_t': '1.95', 'flag': 0}
{'name': 'John', 'surname': 'Darby', 'amount': '980.43', 'delta_t': '7.37', 'flag': 1}
{'name': 'Jill', 'surname': 'Smith', 'amount': '609.90', 'delta_t': '8.34', 'flag': 0}
{'name': 'Andy', 'surname': 'Darby', 'amount': '434.88', 'delta_t': '4.19', 'flag': 1}
{'name': 'John', 'surname': 'Smith', 'amount': '91.56', 'delta_t': '8.53', 'flag': 0}
...



-------------------------------------------
Time: 2023-05-10 07:49:26
-------------------------------------------
{'name': 'John', 'surname': 'Millers', 'amount': '929.58', 'delta_t': '8.73', 'flag': 1}
{'name': 'Joe', 'surname': 'Smith', 'amount': '489.00', 'delta_t': '2.40', 'flag': 0}
{'name': 'Joe', 'surname': 'Darby', 'amount': '322.92', 'delta_t': '4.85', 'flag': 0}
{'name': 'Jill', 'surname': 'Johnson', 'amount': '659.33', 'delta_t': '1.92', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '456.85', 'delta_t': '6.48', 'flag': 0}
{'name': 'Jill', 'surname': 'Jones', 'amount': '205.24', 'delta_t': '1.70', 'flag': 0}
{'name': 'Alice', 'surname': 'Smith', 'amount': '685.04', 'delta_t': '0.61', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '730.81', 'delta_t': '5.03', 'flag': 1}
{'name': 'John', 'surname': 'Jones', 'amount': '44.40', 'delta_t': '4.58', 'flag': 1}
{'name': 'Joe', 'surname': 'Jones', 'amount': '867.80', 'delta_t': '5.06', 'flag': 1}
...



-------------------------------------------
Time: 2023-05-10 07:49:28
-------------------------------------------
{'name': 'Alice', 'surname': 'Johnson', 'amount': '320.71', 'delta_t': '0.53', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '304.54', 'delta_t': '9.74', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '419.29', 'delta_t': '1.55', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '422.52', 'delta_t': '9.17', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '299.17', 'delta_t': '9.25', 'flag': 1}
{'name': 'John', 'surname': 'Millers', 'amount': '328.45', 'delta_t': '6.16', 'flag': 1}
{'name': 'John', 'surname': 'Darby', 'amount': '726.84', 'delta_t': '8.44', 'flag': 1}
{'name': 'Alice', 'surname': 'Jones', 'amount': '753.72', 'delta_t': '8.09', 'flag': 1}
{'name': 'Joe', 'surname': 'Darby', 'amount': '61.51', 'delta_t': '0.64', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '134.77', 'delta_t': '9.16', 'flag': 0}
...



**Start the computations with `ssc.start()` and stop with `ssc.stop(stopSparkContext=False)`.** 

_Remember that once the StreamingContext has been stopped, it must be redefined anew if we want to restart the streaming computations._

In [11]:
ssc.start()

23/05/10 07:49:29 WARN StreamingContext: StreamingContext has already been started


-------------------------------------------
Time: 2023-05-10 07:49:30
-------------------------------------------
{'name': 'Joe', 'surname': 'Darby', 'amount': '238.36', 'delta_t': '2.61', 'flag': 1}
{'name': 'Jill', 'surname': 'Smith', 'amount': '231.05', 'delta_t': '8.40', 'flag': 0}
{'name': 'Alice', 'surname': 'Darby', 'amount': '848.14', 'delta_t': '3.67', 'flag': 1}
{'name': 'John', 'surname': 'Darby', 'amount': '745.50', 'delta_t': '1.06', 'flag': 0}
{'name': 'Jill', 'surname': 'Smith', 'amount': '673.89', 'delta_t': '3.27', 'flag': 0}
{'name': 'Joe', 'surname': 'Smith', 'amount': '460.67', 'delta_t': '0.44', 'flag': 0}
{'name': 'Jill', 'surname': 'Smith', 'amount': '698.89', 'delta_t': '9.11', 'flag': 0}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '260.23', 'delta_t': '7.36', 'flag': 0}
{'name': 'Joe', 'surname': 'Darby', 'amount': '276.52', 'delta_t': '3.05', 'flag': 0}
{'name': 'John', 'surname': 'Millers', 'amount': '405.54', 'delta_t': '9.69', 'flag': 0}
...



-------------------------------------------
Time: 2023-05-10 07:49:32
-------------------------------------------
{'name': 'Joe', 'surname': 'Darby', 'amount': '971.47', 'delta_t': '6.93', 'flag': 0}
{'name': 'John', 'surname': 'Millers', 'amount': '258.15', 'delta_t': '5.12', 'flag': 0}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '45.52', 'delta_t': '3.44', 'flag': 0}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '876.38', 'delta_t': '0.74', 'flag': 0}
{'name': 'Joe', 'surname': 'Darby', 'amount': '678.98', 'delta_t': '5.69', 'flag': 0}
{'name': 'Alice', 'surname': 'Smith', 'amount': '596.28', 'delta_t': '9.84', 'flag': 0}
{'name': 'John', 'surname': 'Darby', 'amount': '752.90', 'delta_t': '2.54', 'flag': 0}
{'name': 'John', 'surname': 'Darby', 'amount': '699.80', 'delta_t': '5.33', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '295.29', 'delta_t': '0.76', 'flag': 1}
{'name': 'Jill', 'surname': 'Darby', 'amount': '52.25', 'delta_t': '7.01', 'flag': 1}
...



-------------------------------------------
Time: 2023-05-10 07:49:34
-------------------------------------------
{'name': 'Andy', 'surname': 'Johnson', 'amount': '15.42', 'delta_t': '1.08', 'flag': 0}
{'name': 'Alice', 'surname': 'Smith', 'amount': '304.55', 'delta_t': '5.75', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '263.64', 'delta_t': '7.70', 'flag': 1}
{'name': 'Joe', 'surname': 'Darby', 'amount': '522.86', 'delta_t': '3.86', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '456.59', 'delta_t': '3.71', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '940.23', 'delta_t': '9.54', 'flag': 1}
{'name': 'Andy', 'surname': 'Darby', 'amount': '624.29', 'delta_t': '1.94', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '340.37', 'delta_t': '9.28', 'flag': 0}
{'name': 'John', 'surname': 'Millers', 'amount': '490.72', 'delta_t': '2.50', 'flag': 0}
{'name': 'John', 'surname': 'Millers', 'amount': '376.87', 'delta_t': '3.99', 'flag': 0}
...



-------------------------------------------
Time: 2023-05-10 07:49:36
-------------------------------------------
{'name': 'Joe', 'surname': 'Smith', 'amount': '471.30', 'delta_t': '7.87', 'flag': 0}
{'name': 'Andy', 'surname': 'Millers', 'amount': '279.48', 'delta_t': '8.78', 'flag': 1}
{'name': 'John', 'surname': 'Smith', 'amount': '287.60', 'delta_t': '1.70', 'flag': 0}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '245.02', 'delta_t': '8.50', 'flag': 0}
{'name': 'Jill', 'surname': 'Smith', 'amount': '481.61', 'delta_t': '0.14', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '645.09', 'delta_t': '0.77', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '58.73', 'delta_t': '7.91', 'flag': 1}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '909.17', 'delta_t': '9.79', 'flag': 1}
{'name': 'Andy', 'surname': 'Millers', 'amount': '714.00', 'delta_t': '3.46', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '321.75', 'delta_t': '1.48', 'flag': 1}
...



...

-------------------------------------------
Time: 2023-05-10 07:50:30
-------------------------------------------
{'name': 'John', 'surname': 'Millers', 'amount': '774.19', 'delta_t': '1.31', 'flag': 1}
{'name': 'Andy', 'surname': 'Millers', 'amount': '741.99', 'delta_t': '9.36', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '961.88', 'delta_t': '3.39', 'flag': 1}
{'name': 'Andy', 'surname': 'Darby', 'amount': '873.07', 'delta_t': '3.22', 'flag': 0}
{'name': 'Jill', 'surname': 'Johnson', 'amount': '37.25', 'delta_t': '3.80', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '225.85', 'delta_t': '6.76', 'flag': 0}
{'name': 'Alice', 'surname': 'Smith', 'amount': '274.15', 'delta_t': '7.76', 'flag': 0}
{'name': 'Joe', 'surname': 'Darby', 'amount': '970.43', 'delta_t': '4.60', 'flag': 1}
{'name': 'Jill', 'surname': 'Jones', 'amount': '94.02', 'delta_t': '5.98', 'flag': 0}
{'name': 'John', 'surname': 'Darby', 'amount': '846.43', 'delta_t': '5.14', 'flag': 0}
...



-------------------------------------------
Time: 2023-05-10 07:50:32
-------------------------------------------
{'name': 'Joe', 'surname': 'Jones', 'amount': '674.50', 'delta_t': '1.33', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '416.16', 'delta_t': '2.26', 'flag': 1}
{'name': 'Andy', 'surname': 'Darby', 'amount': '574.81', 'delta_t': '0.48', 'flag': 0}
{'name': 'John', 'surname': 'Darby', 'amount': '941.08', 'delta_t': '3.24', 'flag': 1}
{'name': 'Andy', 'surname': 'Smith', 'amount': '393.97', 'delta_t': '8.38', 'flag': 0}
{'name': 'Andy', 'surname': 'Darby', 'amount': '212.71', 'delta_t': '8.42', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '602.60', 'delta_t': '4.12', 'flag': 0}
{'name': 'Andy', 'surname': 'Millers', 'amount': '618.76', 'delta_t': '9.51', 'flag': 0}
{'name': 'Alice', 'surname': 'Darby', 'amount': '380.23', 'delta_t': '1.02', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '510.56', 'delta_t': '8.42', 'flag': 0}
...



-------------------------------------------
Time: 2023-05-10 07:50:34
-------------------------------------------
{'name': 'John', 'surname': 'Smith', 'amount': '619.26', 'delta_t': '2.59', 'flag': 0}
{'name': 'John', 'surname': 'Darby', 'amount': '715.60', 'delta_t': '8.02', 'flag': 0}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '419.78', 'delta_t': '0.51', 'flag': 0}
{'name': 'Joe', 'surname': 'Smith', 'amount': '297.08', 'delta_t': '7.07', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '250.02', 'delta_t': '2.57', 'flag': 0}
{'name': 'Joe', 'surname': 'Smith', 'amount': '408.40', 'delta_t': '8.69', 'flag': 0}
{'name': 'Jill', 'surname': 'Johnson', 'amount': '215.53', 'delta_t': '0.35', 'flag': 1}
{'name': 'John', 'surname': 'Jones', 'amount': '103.59', 'delta_t': '9.44', 'flag': 0}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '478.43', 'delta_t': '6.94', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '708.90', 'delta_t': '9.75', 'flag': 0}
...



-------------------------------------------
Time: 2023-05-10 07:50:36
-------------------------------------------
{'name': 'Alice', 'surname': 'Darby', 'amount': '736.54', 'delta_t': '0.24', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '39.62', 'delta_t': '9.62', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '559.65', 'delta_t': '3.31', 'flag': 0}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '206.12', 'delta_t': '2.74', 'flag': 0}
{'name': 'John', 'surname': 'Millers', 'amount': '700.90', 'delta_t': '6.03', 'flag': 0}
{'name': 'Andy', 'surname': 'Darby', 'amount': '577.29', 'delta_t': '8.08', 'flag': 0}
{'name': 'Joe', 'surname': 'Millers', 'amount': '444.45', 'delta_t': '6.47', 'flag': 0}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '532.50', 'delta_t': '9.42', 'flag': 0}
{'name': 'John', 'surname': 'Darby', 'amount': '743.37', 'delta_t': '8.61', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '656.53', 'delta_t': '7.70', 'flag': 0}
...



-------------------------------------------
Time: 2023-05-10 07:50:38
-------------------------------------------
{'name': 'Andy', 'surname': 'Darby', 'amount': '476.14', 'delta_t': '7.50', 'flag': 0}
{'name': 'Jill', 'surname': 'Jones', 'amount': '81.15', 'delta_t': '7.84', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '55.12', 'delta_t': '5.42', 'flag': 0}
{'name': 'Jill', 'surname': 'Millers', 'amount': '809.55', 'delta_t': '2.37', 'flag': 0}
{'name': 'Jill', 'surname': 'Jones', 'amount': '599.12', 'delta_t': '5.77', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '855.22', 'delta_t': '0.56', 'flag': 0}
{'name': 'Jill', 'surname': 'Jones', 'amount': '698.51', 'delta_t': '0.52', 'flag': 0}
{'name': 'Jill', 'surname': 'Millers', 'amount': '335.80', 'delta_t': '6.08', 'flag': 0}
{'name': 'Andy', 'surname': 'Millers', 'amount': '374.03', 'delta_t': '9.25', 'flag': 1}
{'name': 'Alice', 'surname': 'Smith', 'amount': '429.29', 'delta_t': '6.51', 'flag': 0}
...



In [12]:
ssc.stop(stopSparkContext=False)

23/05/10 07:50:40 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver


23/05/10 07:50:40 ERROR TaskSchedulerImpl: Lost executor 1 on 172.21.0.6: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/05/10 07:50:40 WARN TaskSetManager: Lost task 1.0 in stage 227.0 (TID 674) (172.21.0.6 executor 1): ExecutorLostFailure (executor 1 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
-------------------------------------------
Time: 2023-05-10 07:50:40
-------------------------------------------
{'name': 'Jill', 'surname': 'Jones', 'amount': '959.83', 'delta_t': '0.42', 'flag': 1}
{'name': 'Andy', 'surname': 'Smith', 'amount': '895.23', 'delta_t': '8.76', 'flag': 0}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '755.76', 'delta_t': '7.51', 'flag': 0}
{'name': 'Jill', 'surname': 'Jones', 'amount': '962.12', 'delta_t': '6.90', 'flag': 0}
{'nam

## Working with Streaming data

Now that we know how to stream data into Spark, let's explore how we can perform basic distributed operations on the data.

However, before we can proceed, we need to make sure that we have properly restarted the `StreamingContext` object, as the connection between the socket and Spark will be lost when the context is stopped.

To restart the streaming context, we need to:
1. Create a new `StreamingContext` object (we can reuse the `ssc` object in our case).
2. Point it to the correct TCP socket and port where the data is being streamed from.
3. Restart the Python producer application.

Once the `StreamingContext` is properly set up and running, we can start applying distributed operations to the streaming data. 

In [13]:
# create a new Spark StreamingContext with a batch wall-time of 2 seconds
ssc = StreamingContext(sc, 2) 

In [14]:
# define the socket stream using the appropriate endpoint and port
socket_stream = ssc.socketTextStream(hostname, portnumber)

In [15]:
# start the python producer script
### from the terminal/WSL shell

We now start listening on the TCP socket, interpreting the input data stream as `json` loads.

**Remember to get rid of the `pprint()` action, that would otherwise be performed continously, dumping the input data into the Jupyter cells.**

In [16]:
# create a new json_stream object by reading the json loads from the socket
json_stream = socket_stream.map(lambda msg: json.loads(msg))

#### Converting Streaming Data to a DataFrame

To make use of Spark's higher-level APIs, we can convert each batch of streaming data into a DataFrame. 

To do so, we'll first need to convert the numeric features of the incoming JSON data into Python floats and integers. This is a simple type cast operation that can be easily parallelized.

After casting the data, we can create a `Row` object for each transaction using the resulting Python dictionary. These `Row` objects can then be used to create a DataFrame, allowing us to use Spark's higher-level APIs for data processing.

In [17]:
from pyspark.sql import Row

# create a row for each message 
#   convert each numerical value to the proper python type
#   create a row from each message
def create_row_rdd(t):
    t['amount'] = float(t['amount'])
    t['delta_t'] = float(t['delta_t'])
    t['flag'] = int(t['flag'])
    
    return Row(**t)

# apply the transformation to the json_stream rdd
row_stream = json_stream.map(create_row_rdd)

The method `DStream.foreachRDD` can be used to apply custom transformations to each *batch* of data. 

In our case, we are insterested in converting each batch of data into a Spark DataFrame and perform operations, such as counting the number of transactions for each user. 

In this specific use-case, we can identify batches where a user has performed more than one transaction with the `flag` field equal to one as fraudulent. For simplicity, we will assume that these batches represent fraudulent activity.

In reality, this might be a flag you might set on the fly using statig-rules or a ML-based model.

**NOTE**: If left unconstrained, Spark might want to create a very large number of partitions for this streaming application.

Using way more partitions than necessary always results in a huge over-head due to the partition-to-partition communications.

We can force Spark to use a small yet reasonable (given the problem and resources we have) number of partitions
thus making it more efficient in the case of small workloads and few executors

In [18]:
# this line is a trick to force Spark to use a small number of partitions (4 in this example)
spark.conf.set("spark.sql.shuffle.partitions", 4)

### Process each bach to identify possibly fraudolent transactions


1. convert the RDD into a DataFrame (provide the schema if necessary)
2. compute the _number of flagged transactions per batch per user_ (create a unique `userID` field as the combination of _FirstLastname_ to idenfity individual users)
3. identify all the "suspicios" transactions per user: all users with more than one flagged transaction per batch will be assigned a `isFraud` boolean variable
4. format the resulting `userID` and `isFraud` information in a DataFrame to mimick a "live-report" of the suspicious transactions

In [22]:
from pyspark.sql.functions import concat, col, lit, countDistinct

def process_batch(rdd):
    # convert rdd to df
    #   check the documentation and/or the Lecture2 notebook 
    #   for details on how to create and pass a schema to a dataframe   
    df = rdd.toDF(
        schema = 'name string, surname string, amount float, delta_t float, flag int'
    )
    
    # find number of transactions for each user when flag = 1 
    #    declare a new column to create a unique user identifier 
    #    this can be easily done by concatenating first- and last-name fields
    #    check the concat function from pyspark.sql.functions 
    num_transactions = df.where(col('flag')==1)\
                         .withColumn('id', concat(col('name'), col('surname')))\
                         .groupBy('id')\
                         .count()
    
    # find suspicious transactions
    #    filter only users with more than one transaction per batch
    #    create a "fraud" column with a value of 1 for the selected users (check the lit function)
    #    from the dataframe, project only the unique id and fraud columns
    sus_transactions = num_transactions.where(col('count')>1)\
                                        .withColumn('fraud', lit(1))\
                                        .select(col('id'), col('fraud'))
    
    # (trigger an automatic alert)
    # print the first 5 items of the resulting dataframe
    sus_transactions.show(5)

Finally, instruct Spark to execute this `process_batch` function **for each RDD** you will have in your DStream

In [23]:
row_stream.foreachRDD(process_batch)

Now you should be ready to start the spark streaming context

In [24]:
ssc.start()

+---+-----+
| id|fraud|
+---+-----+
+---+-----+



+---+-----+
| id|fraud|
+---+-----+
+---+-----+



+------------+-----+
|          id|fraud|
+------------+-----+
|AliceMillers|    1|
+------------+-----+



+---+-----+
| id|fraud|
+---+-----+
+---+-----+



+-----------+-----+
|         id|fraud|
+-----------+-----+
|AndyMillers|    1|
+-----------+-----+



...

+---+-----+
| id|fraud|
+---+-----+
+---+-----+



+---+-----+
| id|fraud|
+---+-----+
+---+-----+



+---+-----+
| id|fraud|
+---+-----+
+---+-----+



+---+-----+
| id|fraud|
+---+-----+
+---+-----+



+---+-----+
| id|fraud|
+---+-----+
+---+-----+



In [ ]:
# stop streaming context
ssc.stop(stopSparkContext=False)

## Stop worker and master

In [ ]:
sc.stop()
spark.stop()

Finally, use `docker compose down` to stop and clear all running containers.